# Hodgkin-Huxley | Overhead

Compute run times and computational overhead of perturbation

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from itertools import product as itproduct

In [ ]:
from sys import path as sys_path
from os.path import abspath as os_path_abspath
sys_path.append(os_path_abspath('..'))
import addpaths

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import stim_utils
import math_utils
import frame_utils
import metric_utils
import plot_utils as pltu

# Model

In [ ]:
import hodgkin_huxley
t0, tmax = 0, 100
neuron = hodgkin_huxley.neuron()

In [ ]:
stim_onset, stim_offset = 10, tmax-10
stims = [
    stim_utils.Istim(Iamp=0.15, onset=stim_onset, offset=stim_offset, name='Step'),
]
for stim in stims: stim.plot(t0=t0, tmax=tmax)

# Generator

In [ ]:
from data_generator_HH import data_generator_HH
from copy import deepcopy

gens = {}

for stim in stims:
    
    neuron = deepcopy(neuron)
    neuron.get_Istim_at_t = stim.get_I_at_t
    
    gens[stim] = data_generator_HH(
        t0=t0, tmax=tmax, t_eval_adaptive=None, max_step=1.0,
        return_vars=['ys'],
        model=neuron, y0=neuron.compute_yinf(-65), thresh=0.0,
        n_samples=100, n_parallel=20,
        gen_det_sols=False, gen_acc_sols=False,
        base_folder='_data/pert_overhead'
    )
    gens[stim].update_subfoldername(stim=stim.name)

## Data

In [ ]:
# pert_method, adaptive, methods, step_params, pert_params
solver_params = [
    ('conrad', 0, ['FE', 'EE', 'EEMP', 'RKBS', 'RKCK', 'RKDP'], [0.01], [1]),
    ('conrad', 1, ['RKBS', 'RKCK', 'RKDP'], [1e-4], [1]),
    
    ('abdulle', 0, ['FE', 'EE', 'EEMP', 'RKBS', 'RKCK', 'RKDP'], [0.01], [0.1]),    
    ('abdulle', 1, ['RKBS', 'RKCK', 'RKDP'], [1e-4], [0.1]),
    
    ('abdulle_ln', 0, ['FE', 'EE', 'EEMP', 'RKBS', 'RKCK', 'RKDP'], [0.01], [0.1]),    
    ('abdulle_ln', 1, ['RKBS', 'RKCK', 'RKDP'], [1e-4], [0.1]),

    (None, 0, ['FE', 'EE', 'EEMP', 'RKBS', 'RKCK', 'RKDP'], [0.01], ['None']),
    (None, 1, ['RKBS', 'RKCK', 'RKDP'], [1e-4], ['None']),
]

In [ ]:
for stim, gen in gens.items():
    
    print('----------------------------------------------------------')
    print(stim, ':', gen.subfoldername)
    print('----------------------------------------------------------')   
    
    for pert_method, adaptive, methods, step_params, pert_params in solver_params:
        for step_param, method, pert_param in itproduct(step_params, methods, pert_params):
            gen.gen_and_save_data(
                method=method, adaptive=adaptive, step_param=step_param,
                pert_method=pert_method, pert_param=pert_param, allowgenerror=False,
                overwrite=False, 
            )

# Load data

In [ ]:
from data_loader import data_loader

df = pd.DataFrame()

for stim, gen in gens.items():
    stim_df = data_loader(gen).load_data2dataframe(
        solver_params, drop_traces=False, MAEs=False, allowgenerror=True
    )
    metric_utils.add_det_nODEcalls(stim_df, T=gen.tmax-gen.t0)
    stim_df['stimfun'] = stim
    stim_df['stim'] = stim.name
    
    df = df.append(stim_df, ignore_index=True)
    
df.pert_method = df.pert_method.fillna(value='det.')
df = df[['method', 'adaptive', 'pert_method', 'stim', 'run_times']] # Drop some columns

## Sort data

In [ ]:
plot_df = {
    'solver': [], 'stim': [], 'method': [], 'adaptive': [],
    'abdulle_rel_run_times': [],
    'abdulleln_rel_run_times': [],  
    'conrad_rel_run_times': [],
}

for (method, adaptive, stim), group in df.groupby(by=['method', 'adaptive', 'stim']):
    assert group.shape[0] == 4, group.shape[0]
    assert group.pert_method.nunique() == 4   
    
    plot_df['solver'].append(pltu.method2label(method=method, adaptive=adaptive))
    plot_df['method'].append(method)
    plot_df['adaptive'].append(adaptive)
    plot_df['stim'].append(stim)
    
    plot_df['abdulle_rel_run_times'].append(
        group.run_times[group.pert_method == 'abdulle'].iloc[0] / group.run_times[group.pert_method == 'det.'].iloc[0])
    plot_df['abdulleln_rel_run_times'].append(
        group.run_times[group.pert_method == 'abdulle_ln'].iloc[0] / group.run_times[group.pert_method == 'det.'].iloc[0])
    plot_df['conrad_rel_run_times'].append(
        group.run_times[group.pert_method == 'conrad'].iloc[0] / group.run_times[group.pert_method == 'det.'].iloc[0])
    
plot_df = pd.DataFrame(plot_df)

In [ ]:
plot_df.head()

# Plot

## Figure

In [ ]:
fig, axs = pltu.subplots(1,1,ysizerow=1.4,squeeze=False)
pltu.move_xaxis_outward(axs, scale=3)

ax = axs.flat[0]

### Plot data ###
for i, (data, pert_method) in enumerate(zip(
    [plot_df.conrad_rel_run_times, plot_df.abdulle_rel_run_times, plot_df.abdulleln_rel_run_times],
    ['conrad', 'abdulle', 'abdulle_ln']
)):

    positions = pltu.get_x_positions(n_positions=plot_df.shape[0], idx=i, n_idxs=3)

    if pert_method == 'conrad':
        ttl = 'State pert.'
    elif pert_method == 'abdulle':
        ttl = 'Step-size pert. uniform'
    elif pert_method == 'abdulle_ln':
        ttl = 'Step-size pert. log-normal'
    
    pltu.plot_percentiles(
        ax=ax, data=list(data), positions=positions, connect=False,
        color=pltu.neuron2color(i), marker=['X', 'P', 'o'][i],
        mean_kw=dict(label=ttl, ls='None'), showflier=False
    )
    ax.set_xticks(np.arange(plot_df.shape[0]))
    ax.set_xticklabels(list(plot_df.solver), rotation=0)

### Decorate ###
for ax in axs[:,0]: ax.set_ylabel('Rel. run time')
axs.flat[0].legend(loc='upper right', frameon=True, bbox_to_anchor=(1,1.1), borderpad=0.2)
pltu.tight_layout()
sns.despine()
pltu.grid(axs, axis='y')
pltu.savefig('rel_run_times')
pltu.show_saved_figure(fig)

## Text

In [ ]:
for solver, group in plot_df.groupby(['solver'], sort=False):
    method = solver.replace('\mathrm', '').replace('$', '')
    print(f"{method} \t {np.mean(np.concatenate(list(group.conrad_rel_run_times)))*100:.0f}")

In [ ]:
for adaptive, group in plot_df.groupby(['adaptive'], sort=False):
    print(f"{adaptive} \t {np.mean(np.concatenate(list(group.conrad_rel_run_times)))*100:.0f}")

In [ ]:
print(f"{adaptive} \t {np.mean(np.concatenate(list(plot_df.conrad_rel_run_times)))*100:.0f}")

In [ ]:
print(f"{adaptive} \t {np.mean(np.concatenate(list(plot_df.abdulle_rel_run_times)))*100:.0f}")

In [ ]:
print(f"{adaptive} \t {np.mean(np.concatenate(list(plot_df.abdulleln_rel_run_times)))*100:.0f}")

In [ ]:
for solver, group in plot_df.groupby(['solver'], sort=False):
    method = solver.replace('\mathrm', '').replace('$', '')
    print(f"{method} \t {np.mean(np.concatenate(list(group.abdulle_rel_run_times)))*100:.0f}")

In [ ]:
for adaptive, group in plot_df.groupby(['adaptive'], sort=False):
    print(f"{adaptive} \t {np.mean(np.concatenate(list(group.abdulle_rel_run_times)))*100:.0f}")

In [ ]:
for solver, group in plot_df.groupby(['solver'], sort=False):
    method = solver.replace('\mathrm', '').replace('$', '')
    print(f"{method} \t {np.mean(np.concatenate(list(group.abdulleln_rel_run_times)))*100:.0f}")

In [ ]:
for adaptive, group in plot_df.groupby(['adaptive'], sort=False):
    print(f"{adaptive} \t {np.mean(np.concatenate(list(group.abdulleln_rel_run_times)))*100:.0f}")